In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import emcee
import corner
import pickle
from astropy import units as u
from astropy import constants as const

%matplotlib widget

### Load in time series data, use only certain sections

In [2]:
var_path = "example data/tau_0.050"

t_flux = np.load(var_path + "_t.npy")[500:2500]
flux = np.load(var_path + "_f.npy")[500:2500]
flux_err = np.load(var_path + "_ferr.npy")[500:2500]

mu = np.mean(flux)
flux = (flux / mu - 1) * 1e3
flux_err = flux_err * 1e3 / mu

t_rad_full = np.load(var_path + "_t.npy")[700:1750]
rv_full = np.load(var_path + "_rv.npy")[700:1750]
rv_err_full = np.load(var_path + "_rverr.npy")[700:1750]

In [3]:
def time_series_data(save=True, data_dict_fn=None, data_dict=None):
    data_dict_fn = "Two Latent GPs Tests/" + data_dict_fn
    
    if save:
        with open(data_dict_fn, 'wb') as handle:
            pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        with open(data_dict_fn, 'rb') as handle:
            data_dict = pickle.load(handle)
            
        return data_dict
    
#time_series_data(save=False, data_dict_fn='f750-2000_rv800-1750_nrv{0}_seed{1}.pickle'.format(n_rv_cadence, rng_seed), data_dict=trained_data_dic)
#time_series_data(data_dict_fn='f0-1000_rv500-2000_nrv7_equal_spaced.pickle'.format(), data_dict=trained_data_dic)

### Planet Signal Injection

In [4]:
def Newton_Raphson_factor(E0, M, e=0.0):
    top = M - E0 + (e*np.sin(E0))
    bot = 1. - (e * np.cos(E0))

    return top/bot

def Newton_Raphson(M, E0, e=0.0):
    correct = Newton_Raphson_factor(E0, M, e)
    return correct

def inter_NR(M, E0, n, e=0.0):
    E = E0
    for i in range(n):
        E_n = Newton_Raphson_factor(E, M, e)
        E = E_n + E

    return E

def x(a, E, e=0.0):
    return a * (np.cos(E) - e)

def y(a, E, e=0.0):
    return a * np.sqrt(1 - e**2.) * np.sin(E)

def f(E, e=0.0):
    tan_f_2 = np.sqrt((1 + e)/(1 - e)) * np.tan(E/2.)
    f_2 = np.arctan(tan_f_2)
    return f_2 * 2.

def T(a):
    return a**(3./2.)

def M(t, T, tau):
    return ((2.*np.pi) / T) * (t - tau)

In [5]:
"""
What you need to define in order to inject the planet:

1a) semi-major axis (AU) OR
1b) orbital period (year)
2) eccenticity - set to 0.0
3) inclination (rad) - can be taken from the starry configuration
4) m_star (solar mass) - set to 1.0, as in starry
5a) m_planet (jupiter mass) OR
5b) K semiamplitude (m/s)
6) tau0 (year) - set to 0.0 by default
7) argument of periapsis (rad) - set to 0 since eccentricity is set to 0.0
"""

G = const.G
G = G.to(u.AU**3 / (u.Msun * u.yr**2))

def period(a, m2, m1=1.0*u.Msun):
    a *= u.AU
    m2 *= u.Mjup

    top = 4. * np.pi**2. * a**3.
    bot = G * (m1 + m2)

    return np.sqrt(top/bot).to(u.yr).value

def semi_major_axis(P, m2, m1=1.0*u.Msun):
    P *= u.year
    m2 *= u.Mjup

    top = P**2 * G * (m1 + m2)
    bot = 4. * np.pi**2.

    return ((top / bot)**(1/3)).to(u.AU).value

def K(m2, i, m1=1.0*u.Msun, *, e=0.0, T=None, a=None):

    m2 = (m2 * u.Mjup).to(u.Msun)

    if T is None and a:
        a_hold = (((m1)/(m1+m2))*a).value
        T = period(a_hold, m2.value, m1=m1)
    else:
        a_hold = semi_major_axis(T, m2.value) * u.AU
        a_hold = (((m1)/(m1+m2))*a_hold).value
        T = period(a_hold, m2.value, m1=m1)

    T *= u.yr

    top = (2.*np.pi*G)**(1/3) * m2 * np.sin(i)
    bot = T**(1/3) * np.sqrt(1. - e**2.) * m1**(2/3)
    # AU / yr
    return (top/bot).to(u.m / u.s)

def V(semi_amp, true_anom, *, e=0.0, w=0.0, V0=0.):
    return V0 + semi_amp*(np.cos(w + true_anom) + e*np.cos(w))

def planet_model(t, period, semi_amp, tau):
    # Mean anomaly
    M0 = M(t, period, tau)
    # Eccentric anomaly
    E = inter_NR(M=M0, E0=1.0, n=20)
    # True anomaly
    true_anom = f(E)
    
    return V(semi_amp, true_anom) # m/s
    
# I want a HJ: P = 12.5 days, m = 1.5 MJup
planet_period = (9.0 * u.day).to(u.year).value
planet_mass = 1.5 # Jupiter units
planet_t0 = (15.0 * u.day).to(u.year).value # year
sys_inc = np.deg2rad(86.5) # deg

K_planet = K(planet_mass, sys_inc, T=planet_period)
print(K_planet)

planet_V = planet_model((t_rad_full*u.day).to(u.yr).value, planet_period, K_planet.value, planet_t0)

rv_full_activity_only = rv_full.copy()

rv_full += planet_V

146.39347559534244 m / s


In [6]:
fig, ax = plt.subplots(figsize=(14, 4))

ax.scatter(t_rad_full, rv_full, s=7.5, color='black')
ax.plot(t_rad_full, planet_V, color='blue', lw=2.0)
ax.scatter(t_rad_full, rv_full_activity_only, s=2.5, color='orange')

ax.set_xlabel(r"Time (day)")
ax.set_ylabel(r"RV (m s$^{-1}$")

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
#plt.close()

### Get specific data indices

In [8]:
# Pull a random cadence of N~10,25,50
rng_seed = 1001 #404
n_rv_cadence = 50

random = np.random.default_rng(rng_seed)
inds = np.sort(random.choice(np.arange(len(t_rad_full)), size=n_rv_cadence, replace=False))

#inds = np.arange(0, len(t_rad_full), 225)

t_rad = t_rad_full[inds]
rv = rv_full[inds]
rv_err = rv_err_full[inds]

trained_data_dic = {"Flux Time":t_flux, "Flux":flux, "Flux Error":flux_err,
                    "RV Time":t_rad_full, "RV":rv_full, "RV Error":rv_err,
                    "Sampled RV Time":t_rad, "Sampled RV":rv, "Sampled RV Error":rv_err,
                    "RNG Seed":rng_seed, "N RV":n_rv_cadence}

### Definition of the Matern 5/2 kernel and its first + second derivatives

In [9]:
def matern52(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return sigma ** 2 * (1 + x + x ** 2 / 3) * np.exp(-x)

def matern52_cross(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return np.sqrt(5) * sigma ** 2 * np.sign(t1 - t2) * (x + x ** 2) / (3 * rho) * np.exp(-x)

def matern52_grad(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return (5 / 3) * sigma ** 2 * (1 + x - x ** 2) / rho ** 2 * np.exp(-x)

def sample_gp(random, K, size=None):
    return random.multivariate_normal(np.zeros(K.shape[0]), K, size=size)

### Definition of the diagonal and off-diagonal terms for the covariance block matrix

In [10]:
def K_11(t_1, t_2, p):
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
        
    A, B, C, D, lnrho = p
    
    first_term = A**2 * matern52(t_1, t_2, lnrho=lnrho)
    # Note that the 2nd and 3rd terms cancel by a sign
    fourth_term = B**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term
    
def K_12(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = A*C * matern52(t_1, t_2, lnrho=lnrho)
    second_term = A*D * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = B*C * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = B*D * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term

def K_21(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = C*A * matern52(t_1, t_2, lnrho=lnrho)
    second_term = C*B * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = D*A * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = D*B * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term
    
def K_22(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = C**2 * matern52(t_1, t_2, lnrho=lnrho)
    # Note the 2nd and 3rd terms cancel by a sign
    fourth_term = D**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term

In [11]:
def cov_mat(params, t_f, t_rv):
    """
    function to build covariance matrix
    """
    Kappa11 = K_11(t_f, t_f, params)
    Kappa12 = K_12(t_f, t_rv, params)
    Kappa21 = Kappa12.T
    Kappa22 = K_22(t_rv, t_rv, params)

    cov = np.concatenate((
          np.concatenate((Kappa11, Kappa12), axis=1),
          np.concatenate((Kappa21, Kappa22), axis=1),
          ), axis=0)
    
    return cov

In [12]:
def gp_log_like(r, K):
    """
    Pulled from Dan's notebook, updated with Cholesky decomposition
    https://github.com/dfm/gp/blob/main/solutions.ipynb
    
    The multivariate Gaussian ln-likelihood (up to a constant) for the
    vector ``r`` given a covariance matrix ``K``.
    
    :param r: ``(N,)``   The residual vector with ``N`` points.
    :param K: ``(N, N)`` The square (``N x N``) covariance matrix.
    
    :returns lnlike: ``float`` The Gaussian ln-likelihood. 
    """
    # Slow version, factor ~2x slower.
    #return -0.5 * (np.dot(r, np.linalg.solve(K, r)) + np.linalg.slogdet(K)[1])

    # Cholesky decomposition, faster
    # For more info, check out: https://math.stackexchange.com/questions/3158303/using-cholesky-decomposition-to-compute-covariance-matrix-determinant
    try:
        cho_decomp = scipy.linalg.cho_factor(K)
        log_det_cov = 2*np.sum(np.log(np.diag(cho_decomp[0])))
        return -0.5 * (np.dot(r, scipy.linalg.cho_solve(cho_decomp, r)) + log_det_cov) #+ (len(r)*np.log(2.*np.pi)))
    except np.linalg.LinAlgError:
        return -np.inf

In [13]:
def gp_neg_log_prob(params, t_f, t_rv, y, y_err):
    
    jitter = np.exp(params[0])
    period = np.exp(params[1])
    semi_amp = np.exp(params[2])
    tau = params[3]
    kernel_params = params[4:]
    kernel_params[0] = np.exp(kernel_params[0])
    
    planet = planet_model((t_rv*u.day).to(u.year).value, (period*u.day).to(u.year).value, semi_amp, (tau*u.day).to(u.year).value)
    
    # y - mod (only RVs)
    ymmod = y.copy()
    ymmod[len(t_f):] -= planet
    
    # Compute the covariance matrix for the first GP
    K1 = cov_mat(kernel_params[:5], t_f, t_rv)
    
    # Compute the covariance matrix for the second GP
    K2 = cov_mat(kernel_params[5:], t_f, t_rv)
    
    K = K1 + K2
    K[np.diag_indices_from(K)] += y_err**2 + jitter
    
    # Compute the negative log likelihood
    return -gp_log_like(ymmod, K)

### Optimize model parameters and Sample Parameters

In [14]:
def minimize_gp_kernel(y):
    
    #opt_kernel_params = np.array([#-1.15102806e+01,
    #                     2.00000000e+00, -1.29627264e-02, 1.90738599e+01, -5.51867547e+01, 9.19003160e-01,
    #                     2.51783582e-02, 4.64294260e-03, 2.33719347e-01, 1.98977475e+01, 3.07601905e-01])
    
    p0 = np.array([np.log(0.9**2.), np.log(11.0), np.log(180.0), 16.0,
                   np.log(0.5), -0.4, 0.7, 5.0, np.log(2.3),
                   0.5, -0.4, 0.7, 5.0, np.log(2.3)])
    
    b = [(np.log(1e-3**2), np.log(1e2**2)), np.log((5.0, 50.0)), np.log((100.0, 250.0)), (0.0, 50.0),
         np.log((1e-3, 300.0)), (-300.0, 300.0), (-300.0, 300.0), (-300.0, 300.0), np.log((0.1, 5)),
         (-300.0, 300.0), (-300.0, 300.0), (-300.0, 300.0), (-300.0, 300.0), np.log((0.1, 5))]
    
    #options = {'maxiter':25000}
    #result = scipy.optimize.minimize(gp_neg_log_prob, p0, args=(t_flux, t_rad, y, np.concatenate((flux_err, rv_err))), method='Nelder-Mead', options={'maxiter':25000}, bounds=b)
    
    result = scipy.optimize.minimize(gp_neg_log_prob, p0, args=(t_flux, t_rad, y, np.concatenate((flux_err, rv_err))), bounds=b)

    return result

In [15]:
y = np.concatenate((flux, rv))
res = minimize_gp_kernel(y)

print(res)
print(res.x)

      fun: -524.5152517580124
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.38556390e+01, -7.48228790e+00,  2.32602135e+00,  1.61294338e+00,
       -1.90156471e+01,  1.29195098e+01,  3.89149737e-02, -3.74586464e-01,
       -1.01593044e+03, -1.38851423e+01, -6.08841418e+00,  4.75938200e-01,
       -8.19454076e-02, -1.03983082e+03])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 840
      nit: 12
     njev: 56
   status: 0
  success: True
        x: array([ -13.81478774,    2.09083296,    4.60517019,    0.        ,
         -6.85366392,  299.66650442,  299.81732688,  299.9670042 ,
          1.60943791, -299.99923299,  299.66005957,  299.74900444,
        299.80195556,    1.60943791])
[ -13.81478774    2.09083296    4.60517019    0.           -6.85366392
  299.66650442  299.81732688  299.9670042     1.60943791 -299.99923299
  299.66005957  299.74900444  299.80195556    1.60943791]


In [16]:
print(res.x[0])
print(res.x[1:4])
print(res.x[4:8])
print(res.x[8])
print(res.x[9:13])
print(res.x[13])

-13.81478773977623
[2.09083296 4.60517019 0.        ]
[ -6.85366392 299.66650442 299.81732688 299.9670042 ]
1.6094379124341003
[-299.99923299  299.66005957  299.74900444  299.80195556]
1.6094379124341003


### Sample after optimization

In [17]:
def gp_log_prob(params, t_f, t_rv, y, y_err):
    """
    add in mean flux and mean RVs as potential parameters in the model?
    mean is weird?
    
    params should be array-like that looks like:
    
    [mean(flux), mean(rv), log(jitter), A, B, C, D, log(rho_1), E, F, G, H, log(rho_2)]
    
    Hard uniform bounds for coefficients
    """
    
    b = [(np.log(1e-3**2), np.log(1e2**2)), (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),
         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]
    
    for b, p in zip(bounds, params):
        if p < b[0] or p > b[-1]:
            return -np.inf
    
    jitter = np.exp(params[0])
    kernel_params = params[1:]
    
    # Compute the covariance matrix for the first GP
    K1 = cov_mat(kernel_params[:5], t_f, t_rv)
    
    # Compute the covariance matrix for the second GP
    K2 = cov_mat(kernel_params[5:], t_f, t_rv)
    
    K = K1 + K2
    K[np.diag_indices_from(K)] += y_err**2 + jitter
    
    # Compute the log likelihood
    return gp_log_like(y, K)

"""
bounds = [(np.log(1e-3**2), np.log(1e2**2)), (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),
         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]

ndim = 5*2 + 1
nwalkers = 250
pos = res.x

pos_fixed = []

for b, p in zip(bounds, pos):
    if p < b[0] + 0.1:
        pos_fixed.append(p + 0.5)
    elif p > b[-1] - 0.1:
        pos_fixed.append(p - 0.5)
    else:
        pos_fixed.append(p)

pos = pos + 1e-3*np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, gp_log_prob,
                                args=(t_flux, t_rad, np.concatenate((flux, rv)), np.concatenate((flux_err, rv_err))))
"""
#sampler.run_mcmc(pos, 1000, progress=True);

'\nbounds = [(np.log(1e-3**2), np.log(1e2**2)), (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),\n         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]\n\nndim = 5*2 + 1\nnwalkers = 250\npos = res.x\n\npos_fixed = []\n\nfor b, p in zip(bounds, pos):\n    if p < b[0] + 0.1:\n        pos_fixed.append(p + 0.5)\n    elif p > b[-1] - 0.1:\n        pos_fixed.append(p - 0.5)\n    else:\n        pos_fixed.append(p)\n\npos = pos + 1e-3*np.random.randn(nwalkers, ndim)\nsampler = emcee.EnsembleSampler(nwalkers, ndim, gp_log_prob,\n                                args=(t_flux, t_rad, np.concatenate((flux, rv)), np.concatenate((flux_err, rv_err))))\n'

In [18]:
l = ["log(jitter)", "$P_b$", "$K_b$", "A", "B", "C", "D", r"log$(\rho_1)$",
     "E", "F", "G", "H", r"log$(\rho_2)$"]

#np.save("example data/two_latent_GP_actual_data_samples.npy", sampler.flatchain)
#np.save("example data/two_latent_GP_mcmc_samples_full_model_n7500_optimized.npy", sampler.flatchain)

#corner.corner(sampler.flatchain[-1000:], labels=l)

#corner.corner(samples, labels=["A", "B", "C", "D", r"$\sigma$", r"$\rho$"], truths=[1.0, -5.0, 0.9, 12.5, np.log(4.0), np.log(2.5)])

In [19]:
#samples = np.load("example data/two_latent_GP_mcmc_samples_full_model_n7500_optimized.npy")
#corner.corner(samples[25000:], labels=l, truths=p_test)

### Plot model and data

In [20]:
def cov_mat_test(params, t_test_f, t_test_rv, t_train_f, t_train_rv):
    """
    function to build covariance matrix for test data
    """
    
    #params = params[3:]
    
    Kappa11 = K_11(t_test_f, t_train_f, params)
    Kappa12 = K_12(t_test_f, t_train_rv, params)
    Kappa21 = K_21(t_test_rv, t_train_f, params)
    Kappa22 = K_22(t_test_rv, t_train_rv, params)
            
    cov = np.concatenate((
          np.concatenate((Kappa11, Kappa12), axis=1),
          np.concatenate((Kappa21, Kappa22), axis=1),
          ), axis=0)
    
    return cov

def trained_cov(p):
    """
    function to build covariance matrix from optimized model parameters computed from training set 
    """
    cov_train1 = cov_mat(p[1:6], t_flux, t_rad)
    cov_train2 = cov_mat(p[6:], t_flux, t_rad)
    cov_train = cov_train1 + cov_train2

    cov_train[np.diag_indices_from(cov_train)] += np.concatenate((flux_err, rv_err))**2 + np.exp(p[0])
    
    return cov_train

def predict(p, y, n_test=1000):
    
    p[0] = np.exp(p[0])
    
    cov_train = trained_cov(p)
    
    factor = (scipy.linalg.cholesky(cov_train, overwrite_a=True, lower=False), False)
    alpha  = scipy.linalg.cho_solve(factor, y, overwrite_b=True)
    
    t_test_flux = np.linspace(min(t_flux), max(t_flux), n_test)
    t_test_rv  = np.linspace(min(t_rad_full), max(t_rad_full), n_test)
    
    cov_test_only1 = cov_mat(p[1:6], t_test_flux, t_test_rv)
    cov_test_only2 = cov_mat(p[6:],  t_test_flux, t_test_rv)
    cov_test_only = cov_test_only1 + cov_test_only2
    
    cov_test1 = cov_mat_test(p[1:6], t_test_flux, t_test_rv, t_flux, t_rad) # t_flux and t_rad are training data
    cov_test2 = cov_mat_test(p[6:],  t_test_flux, t_test_rv, t_flux, t_rad)
    cov_test = cov_test1 + cov_test2
    
    mu = np.dot(cov_test, alpha)
    var = cov_test_only[np.diag_indices_from(cov_test_only)]
    inv_cov_test = np.linalg.solve(cov_train, cov_test.T)
    var -= np.sum(cov_test.T * inv_cov_test, axis = 0)
    
    return mu, var, t_test_flux, t_test_rv

In [21]:
#p0 = res.x
#mu, var = predict(alpha, p0, cov_train, t_test_flux, t_test_rv, t_flux, t_rad)
#mu, var, t_test_flux, t_test_rv = predict(res.x)

#opt_kern_params = np.array([2.00000000e+00, -1.29627264e-02, 1.90738599e+01, -5.51867547e+01, 9.19003160e-01,
#                            2.51783582e-02, 4.64294260e-03, 2.33719347e-01, 1.98977475e+01, 3.07601905e-01])

recovered_planet = planet_model((t_rad*u.day).to(u.yr).value, (np.exp(res.x[1]) * u.day).to(u.year).value, np.exp(res.x[2]), (res.x[3]* u.day).to(u.year).value)
opt_kern_params = [res.x[0]] + list(res.x[4:])

ymmod = y.copy()
ymmod[len(t_flux):] -= recovered_planet

mu, var, t_test_flux, t_test_rv = predict(opt_kern_params, ymmod)

In [22]:
print(mu)

[ -3.86570305  -4.07715773  -4.14676243 ... -37.07092272 -38.18406479
 -39.27078402]


In [23]:
def plot_data_and_model():
    
    # We'll use two separate gridspecs to have different margins, hspace, etc
    gs_top = plt.GridSpec(4, 1, top=0.95)
    gs_base = plt.GridSpec(4, 1, hspace=0)
    fig = plt.figure(figsize=(12, 10))
    
    # Top (unshared) axes
    lc_ax = fig.add_subplot(gs_top[0,:])
    
    # The four shared axes
    rv_ax = fig.add_subplot(gs_base[1,:]) # Need to create the first one to share...
    other_axes = [fig.add_subplot(gs_base[i,:], sharex=rv_ax, sharey=rv_ax) for i in range(2, 4)]
    bottom_axes = [rv_ax] + other_axes
    
    # Hide shared x-tick labels
    for ax in bottom_axes[:-1]:
        plt.setp(ax.get_xticklabels(), visible=False)
    
    #fig, ax = plt.subplots(nrows=3, figsize=(12, 10), gridspec_kw={'hspace':[0.4, 0.0]})
    
    lc_ax.scatter(t_flux, flux, color='black', s=5.0, label='LC w/ errors')#, alpha=0.5)
    lc_ax.plot(t_flux, flux-flux_err, color='grey', label='True LC', zorder=9)#, alpha=0.5)
    lc_ax.plot(t_test_flux, mu[:len(t_test_flux)], lw=2.0, color='C0', ls='-', label='GP$_\mathrm{LC}$', zorder=10)
    
    lc_ax.fill_between(t_test_flux, y1=mu[:len(t_test_flux)]-np.sqrt(var[:len(t_test_flux)]),
                   y2=mu[:len(t_test_flux)]+np.sqrt(var[:len(t_test_flux)]), color='C0', alpha=0.5)
    
    plan_mod = planet_model((t_test_rv*u.day).to(u.yr).value, (np.exp(res.x[1]) * u.day).to(u.year).value, np.exp(res.x[2]), (res.x[3]* u.day).to(u.year).value)
    
    rv_train = rv_ax.scatter(t_rad, rv, color='orange', s=100.0, alpha=0.9, zorder=10, marker='*', label='Trained RVs')
    rv_data = rv_ax.scatter(t_rad_full, rv_full, color='black', s=5.0, label='RVs w/ errors')#, alpha=0.3)
    true_rv, = rv_ax.plot(t_rad_full, rv_full-rv_err_full, color='grey', label='True RVs')#, alpha=0.3)
    full_model, = rv_ax.plot(t_test_rv, mu[len(t_test_flux):]+plan_mod, color='pink', lw=2.0, label='Full Model')
    rv_ax.fill_between(t_test_rv, y1=mu[len(t_test_flux):]-np.sqrt(var[len(t_test_flux):])+plan_mod,
                       y2=mu[len(t_test_flux):]+np.sqrt(var[len(t_test_flux):])+plan_mod, color='pink', alpha=0.25)
    
    rv_gp, = bottom_axes[1].plot(t_test_rv, mu[len(t_test_flux):], lw=2.0, color='C0', ls='-', zorder=1, label='GP$_\mathrm{RV}$')
    act = bottom_axes[1].scatter(t_rad_full, rv_full_activity_only, color='grey', label='Activity Data', s=5.0)
    bottom_axes[1].fill_between(t_test_rv, y1=mu[len(t_test_flux):]-np.sqrt(var[len(t_test_flux):]),
                       y2=mu[len(t_test_flux):]+np.sqrt(var[len(t_test_flux):]), color='C0', alpha=0.25)      
    
    rec_plan, = bottom_axes[2].plot(t_test_rv, plan_mod, color='maroon', lw=4.0, alpha=0.25, label='Recovered Planet')
    tru_plan, = bottom_axes[2].plot(t_rad_full, planet_V, color='grey', lw=2.5, label='True, Injected Planet')
    
    lc_ax.set_ylabel(r"Light Curve (ppt)", fontsize=19)
    rv_ax.set_ylabel(r"Total RV (m s$^{-1}$)", fontsize=18)
    other_axes[0].set_ylabel(r"Activity (m s$^{-1}$)", fontsize=18)
    other_axes[1].set_ylabel(r"Planet (m s$^{-1}$)", fontsize=18)
    
    lc_ax.set_xlabel(r"Time (day)", fontsize=20)
    other_axes[-1].set_xlabel(r"Time (day)", fontsize=20)
    
    lc_ax.set_xlim([min(t_flux), max(t_flux)])
    other_axes[-1].set_xlim([min(t_rad), max(t_rad)])
    
    lc_ax.set_ylim([min(flux)-2.0, max(flux)+2.5])
    other_axes[-1].set_ylim([min(rv_full)-175., max(rv_full)+150.])
    
    for ax in bottom_axes:
        ax.margins(0.05)
    
    lc_ax.legend(fontsize=14, markerscale=2.0)
    rv_p1 = rv_ax.legend([true_rv, full_model], ["True RV", "Total Model"],
                         fontsize=14, markerscale=1.5, bbox_to_anchor=(0.115,0.96), loc="upper left")
    rv_l1 = rv_ax.legend([rv_train, rv_data], ["Trained RVs", "RVs w/ errors"], fontsize=14, markerscale=1.5,
                         bbox_to_anchor=(0.525,0.475), loc="upper right")
    bottom_axes[1].legend([act, rv_gp], ["Activity", "GP$_\mathrm{RV}$"], fontsize=14, markerscale=1.5)
    bottom_axes[2].legend(fontsize=14, markerscale=1.5)
    rv_ax.add_artist(rv_p1)

    data_dict_fn = "Two Latent GPs Tests/Matern-52/"
    plot_fn = data_dict_fn + 'f500-2500_rv700-1750_nrv{0}_seed{1}.png'.format(n_rv_cadence, rng_seed)
    #plot_fn = data_dict_fn + 'f0-1000_rv2500-3500_nrv7_equal_spaced.png'
    #plt.savefig(plot_fn, bbox_inches='tight', dpi=400)
    #plt.savefig("Plots/two_latent_GP_model_comp_nRV_10.png", bbox_inches='tight', dpi=400)
    
    #plt.tight_layout()
    plt.show()

plot_data_and_model()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
#plt.close()